In [0]:
%sql
CREATE CATALOG IF NOT EXISTS dev
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragechhosqejmo4lc.dfs.core.windows.net/613276187473539';

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dev.demodb



In [0]:
%sql
DROP DATABASE IF EXISTS dev.default CASCADE


In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""


flight_time_dt = spark.read.format("json").schema(flight_schema_ddl).option("dateFormat", "M/d/y").load("abfss://dbfs-container@paschaldubem.dfs.core.windows.net/external/flight-time.json")

In [0]:
display(flight_time_dt)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.demodb.flight_time_dt (
  FL_DATE date,
  OP_CARRIER string,
  OP_CARRIER_FL_NUM integer,
  ORIGIN string,
  ORIGIN_CITY_NAME string,
  DEST string,
  DEST_CITY_NAME string, 
  CRS_DEP_TIME INT, 
  DEP_TIME INT, 
  WHEELS_ON INT, 
  TAXI_IN INT, 
  CRS_ARR_TIME INT, 
  ARR_TIME INT, 
  CANCELLED STRING, 
  DISTANCE INT
)

USING DELTA

In [0]:
flight_time_dt.write.format("delta").mode("append").saveAsTable("dev.demodb.flight_time_tbl")

In [0]:
flight_time_tbl = spark.read.table("dev.demodb.flight_time_tbl")
display(flight_time_tbl)

SAVE DELTA DATA IN EXTERNAL LOCATION

In [0]:
(flight_time_tbl
    .write.format("delta")
    .mode("overwrite")
    .save("abfss://dbfs-container@paschaldubem.dfs.core.windows.net/external/flight-time")
)

In [0]:

people_df = spark.read.format("json").load("abfss://dbfs-container@paschaldubem.dfs.core.windows.net/external/people.json")
display(people_df)

DELETE FROM A TABLE 

In [0]:
people_df.write.format("delta").mode("overwrite").saveAsTable("dev.demodb.people")
people = spark.read.table("dev.demodb.people")
display(people)

In [0]:
%sql
-- deelete from dev.demodb.people where frame = 'abdul'
    

DELETE FROM dev.demodb.people
WHERE fname = 'abdul';

SELECT * FROM dev.demodb.people


In [0]:
%sql
UPDATE dev.demodb.people SET dob = "1975-05-25" WHERE dob = "1975-05-25";
    
SELECT * FROM dev.demodb.people

TIME TRAVEL and CHECK PREVIOUS VERSIONS

In [0]:
%sql
describe history dev.demodb.people

In [0]:
%sql
SELECT * FROM dev.demodb.people VERSION AS OF 3

In [0]:
%sql
SELECT * FROM dev.demodb.people TIMESTAMP AS OF "2025-06-22T15:55:03.000+00:00"

ROLL BACK OR WHEN WE DELETE OUR TABLE WHAT CAN WE DO?

In [0]:
%sql
DELETE FROM dev.demodb.people 

In [0]:
%sql
SELECT * FROM dev.demodb.people

REESTOR DELETED TABLE FROM VERSION HISTORY

In [0]:
%sql
describe history dev.demodb.people

In [0]:
%sql
restore table dev.demodb.people to version as of 3;
SELECT * FROM dev.demodb.people
    

